# Hand posture recognition system
- 재활운동에서 센서 기반의 팔운동 재활자세 인식 시스템
- 동작 한번에 1초에 데이터 20개씩 5초간 총 100개의 roll-pitch-yaw 데이터를 얻음. 잘된 자세로 50회 반복, 잘못된 자세를 50회 반복한 데이터
- Coded by vislab, PNU
- 2024/11/29

In [1]:
import numpy as np
import os

In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import pdb


from matplotlib import font_manager, rc
font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family=font_name)
plt.rcParams['axes.unicode_minus'] = False

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset

In [36]:
# import pickle
# from skorch import NeuralNetRegressor
from skorch import NeuralNetClassifier
from skorch.callbacks import Checkpoint
from skorch.helper import predefined_split

from typing import Iterable

In [6]:
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

torch.cuda.set_device(0)
torch.manual_seed(20201130)
torch.cuda.manual_seed(20201130)

### Data read

In [37]:
folder_path = os.path.join('./data')
npy_files = [file for file in os.listdir(folder_path) if file.endswith('.npy')]
all_data = [np.load(os.path.join(folder_path, file), allow_pickle=True) for file in npy_files]
all_data[0].shape, all_data[1].shape

((50, 100, 3), (50, 100, 3))

### Data normalization

In [8]:
def my_normalize(train_all):
    maximum = np.ones(train_all.shape[2])
    minimum = np.ones(train_all.shape[2])
    
    for v in range(train_all.shape[2]):
        maximum[v] = np.max(train_all[:,:,v])
        minimum[v] = np.min(train_all[:,:,v])            

        denom = maximum[v]-minimum[v]
        train_all[:,:,v] = np.clip((train_all[:,:,v]-minimum[v]) / denom, 0.0, 1.0)
    
    print(maximum, minimum)
    return train_all[:50], train_all[50:]


train_all = np.concatenate(all_data, 0)
train_c0, train_c1 = my_normalize(train_all)
train_c0.shape, train_c1.shape

[210.32534281  17.03424847 161.64344788] [-196.39190628  -94.25695038 -170.96961975]


((50, 100, 3), (50, 100, 3))

### Data extraction 

In [9]:
def extract_crop(indata, classes=0):
    crops = []
    for data in indata:
        for ii in range(64,100):
            crops.append(data[ii-64:ii])

    crops = np.array(crops, dtype=np.float32)
    return crops, np.full(crops.shape[0], classes, dtype=int)


train_x0, train_y0 = extract_crop(train_c0[:40], 0)
valid_x0, valid_y0 = extract_crop(train_c0[40:], 0)
train_x1, train_y1 = extract_crop(train_c1[:40], 1)
valid_x1, valid_y1 = extract_crop(train_c1[40:], 1)

train_x, train_y = np.concatenate((train_x0,train_x1),0), np.concatenate((train_y0,train_y1),0)
valid_x, valid_y = np.concatenate((valid_x0,valid_x1),0), np.concatenate((valid_y0,valid_y1),0)

train_x.shape, train_y.shape, valid_x.shape, valid_y.shape

((2880, 64, 3), (2880,), (720, 64, 3), (720,))

In [10]:
# PyTorch TensorDataset으로 변환
train_dataset = TensorDataset(torch.tensor(train_x), torch.tensor(train_y, dtype=torch.long))
valid_dataset = TensorDataset(torch.tensor(valid_x), torch.tensor(valid_y, dtype=torch.long))

### Model

In [17]:
class MotionClassifier(nn.Module):
    def __init__(self, tlen=64):
        super(MotionClassifier, self).__init__()
        self.gru1 = nn.GRU(3, 32, 1, batch_first=True)   
        self.fc1 = nn.Linear(2048, 128)   
        self.fc2 = nn.Linear(128, 2)   
        
    def forward(self, x):
        r1 = torch.randn(1, x.shape[0], 32).to(x.device)  # (1, batch_size, hidden_size)
        t1, _ = self.gru1(x, r1)  # (batch_size, sequence_length, hidden_size)

        o1 = self.fc1(t1.reshape(t1.shape[0], -1)) 
        o2 = self.fc2(F.softplus(o1))  

        return F.softmax(o2, dim=1)

    
# 모델 초기화
model = MotionClassifier()

### Skorch Regressor

In [18]:
net = NeuralNetClassifier(
    model,
    max_epochs=10000,  # 50
    lr=0.0001,
    optimizer=torch.optim.Adam,
    iterator_train__shuffle=True,
    batch_size=1024,
    train_split=predefined_split(valid_dataset),
    callbacks=[Checkpoint(f_params='best_params.pt')],
    device = 'cuda' if torch.cuda.is_available() else 'cpu' 
)

In [19]:
net.fit(train_dataset, y=None) 

  epoch    train_loss    valid_acc    valid_loss    cp     dur
-------  ------------  -----------  ------------  ----  ------
      1        0.6965       0.5653        0.6891     +  0.2137
      2        0.6857       0.5000        0.6884     +  0.0530
      3        0.6820       0.4986        0.6839     +  0.0500
      4        0.6748       0.5986        0.6788     +  0.0510
      5        0.6677       0.7472        0.6753     +  0.0550
      6        0.6617       0.7514        0.6714     +  0.0520
      7        0.6550       0.7014        0.6666     +  0.0521
      8        0.6477       0.6389        0.6617     +  0.0530
      9        0.6411       0.6278        0.6582     +  0.0530
     10        0.6341       0.6389        0.6531     +  0.0520
     11        0.6268       0.6667        0.6495     +  0.0530
     12        0.6196       0.7028        0.6454     +  0.0530
     13        0.6116       0.7000        0.6413     +  0.0520
     14        0.6039       0.6806        0.6355     + 

     99        0.1008       0.9292        0.2706        0.0610
    100        0.0999       0.9278        0.2672        0.0620
    101        0.0998       0.9292        0.2715        0.0510
    102        0.1000       0.9278        0.2650     +  0.0510
    103        0.0979       0.9319        0.2730        0.0500
    104        0.0984       0.9292        0.2700        0.0600
    105        0.0968       0.9278        0.2655        0.0560
    106        0.0961       0.9306        0.2733        0.0583
    107        0.0953       0.9333        0.2726        0.0490
    108        0.0958       0.9306        0.2675        0.0480
    109        0.0946       0.9292        0.2662        0.1290
    110        0.0933       0.9306        0.2714        0.0480
    111        0.0948       0.9278        0.2679        0.0480
    112        0.0935       0.9319        0.2632     +  0.0470
    113        0.0921       0.9264        0.2688        0.0480
    114        0.0909       0.9319        0.2682       

    213        0.0557       0.9444        0.2302     +  0.0500
    214        0.0548       0.9472        0.2360        0.0520
    215        0.0547       0.9486        0.2390        0.0520
    216        0.0530       0.9403        0.2257     +  0.0500
    217        0.0541       0.9472        0.2300        0.0500
    218        0.0537       0.9472        0.2318        0.0500
    219        0.0526       0.9417        0.2270        0.0510
    220        0.0525       0.9472        0.2261        0.0490
    221        0.0522       0.9486        0.2339        0.1430
    222        0.0521       0.9444        0.2279        0.0540
    223        0.0522       0.9431        0.2243     +  0.0500
    224        0.0517       0.9472        0.2343        0.0560
    225        0.0506       0.9528        0.2256        0.0520
    226        0.0519       0.9472        0.2247        0.0550
    227        0.0510       0.9444        0.2277        0.0530
    228        0.0495       0.9458        0.2291       

    329        0.0293       0.9583        0.1764        0.0530
    330        0.0298       0.9597        0.1733        0.0500
    331        0.0296       0.9597        0.1668     +  0.0510
    332        0.0286       0.9583        0.1775        0.0550
    333        0.0308       0.9583        0.1754        0.0520
    334        0.0288       0.9625        0.1652     +  0.0510
    335        0.0293       0.9597        0.1711        0.1310
    336        0.0291       0.9597        0.1764        0.0520
    337        0.0288       0.9625        0.1642     +  0.0510
    338        0.0291       0.9583        0.1703        0.0520
    339        0.0282       0.9625        0.1731        0.0500
    340        0.0278       0.9597        0.1688        0.0490
    341        0.0281       0.9653        0.1626     +  0.0490
    342        0.0275       0.9625        0.1660        0.0510
    343        0.0278       0.9625        0.1650        0.0500
    344        0.0271       0.9653        0.1616     + 

    448        0.0161       0.9722        0.1186        0.0500
    449        0.0160       0.9708        0.1256        0.1380
    450        0.0155       0.9750        0.1214        0.0525
    451        0.0154       0.9750        0.1117     +  0.0510
    452        0.0157       0.9708        0.1214        0.0510
    453        0.0155       0.9736        0.1149        0.0550
    454        0.0149       0.9764        0.1090     +  0.0530
    455        0.0159       0.9722        0.1169        0.0510
    456        0.0153       0.9750        0.1081     +  0.0480
    457        0.0153       0.9764        0.1088        0.0520
    458        0.0155       0.9736        0.1206        0.0540
    459        0.0156       0.9722        0.1158        0.0540
    460        0.0156       0.9750        0.1072     +  0.0503
    461        0.0147       0.9736        0.1136        0.0500
    462        0.0147       0.9750        0.1152        0.0510
    463        0.0147       0.9764        0.1108       

    571        0.0086       0.9819        0.0834        0.0510
    572        0.0085       0.9806        0.0817     +  0.0510
    573        0.0090       0.9778        0.0900        0.0510
    574        0.0093       0.9806        0.0839        0.0520
    575        0.0086       0.9806        0.0900        0.0510
    576        0.0092       0.9806        0.0818        0.0530
    577        0.0091       0.9806        0.0889        0.0590
    578        0.0086       0.9833        0.0851        0.0540
    579        0.0084       0.9806        0.0856        0.0570
    580        0.0089       0.9819        0.0820        0.0540
    581        0.0086       0.9819        0.0857        0.0580
    582        0.0089       0.9806        0.0841        0.0540
    583        0.0092       0.9819        0.0857        0.0600
    584        0.0082       0.9806        0.0887        0.0590
    585        0.0081       0.9806        0.0830        0.0580
    586        0.0087       0.9833        0.0835       

    696        0.0059       0.9806        0.0751        0.0470
    697        0.0055       0.9847        0.0697        0.0500
    698        0.0056       0.9819        0.0795        0.0510
    699        0.0062       0.9847        0.0732        0.1430
    700        0.0058       0.9819        0.0690        0.0500
    701        0.0051       0.9819        0.0782        0.0500
    702        0.0053       0.9819        0.0753        0.0490
    703        0.0059       0.9833        0.0729        0.0510
    704        0.0059       0.9847        0.0722        0.0530
    705        0.0055       0.9833        0.0771        0.0500
    706        0.0057       0.9806        0.0846        0.0530
    707        0.0054       0.9819        0.0748        0.0500
    708        0.0057       0.9847        0.0711        0.0480
    709        0.0059       0.9819        0.0733        0.0480
    710        0.0054       0.9819        0.0828        0.0510
    711        0.0059       0.9819        0.0719       

    825        0.0039       0.9833        0.0695        0.0500
    826        0.0041       0.9833        0.0716        0.0510
    827        0.0044       0.9833        0.0708        0.0510
    828        0.0041       0.9847        0.0703        0.0520
    829        0.0041       0.9861        0.0707        0.0510
    830        0.0044       0.9847        0.0679        0.1430
    831        0.0041       0.9833        0.0732        0.0550
    832        0.0042       0.9847        0.0754        0.0510
    833        0.0044       0.9847        0.0706        0.0510
    834        0.0038       0.9847        0.0694        0.0500
    835        0.0039       0.9847        0.0666        0.0490
    836        0.0043       0.9833        0.0720        0.0510
    837        0.0041       0.9833        0.0782        0.0500
    838        0.0042       0.9833        0.0706        0.0510
    839        0.0044       0.9861        0.0709        0.0490
    840        0.0037       0.9847        0.0749       

    953        0.0031       0.9847        0.0764        0.0480
    954        0.0032       0.9861        0.0755        0.0520
    955        0.0035       0.9847        0.0703        0.0550
    956        0.0032       0.9847        0.0679        0.0530
    957        0.0031       0.9847        0.0754        0.0500
    958        0.0030       0.9847        0.0752        0.0510
    959        0.0034       0.9861        0.0766        0.0550
    960        0.0033       0.9875        0.0689        0.0535
    961        0.0033       0.9861        0.0715        0.0500
    962        0.0030       0.9833        0.0791        0.0530
    963        0.0034       0.9833        0.0800        0.0550
    964        0.0029       0.9861        0.0691        0.0510
    965        0.0030       0.9875        0.0717        0.1320
    966        0.0031       0.9861        0.0734        0.0490
    967        0.0033       0.9875        0.0752        0.0490
    968        0.0026       0.9861        0.0747       

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=MotionClassifier(
    (gru1): GRU(3, 32, batch_first=True)
    (fc1): Linear(in_features=2048, out_features=128, bias=True)
    (fc2): Linear(in_features=128, out_features=2, bias=True)
  ),
)

In [28]:
# 학습된 pt파일을 올리기
#net.load_params(f_params='best_params_0986.pt')
torch.save(model.state_dict(), "best_params_0986.pt")

In [29]:
# 예측 테스트
res = net.predict(valid_dataset)
#res = net.predict(valid_x[:10])
res

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [35]:
res = model(torch.tensor(valid_x[:10],dtype=torch.float).to('cuda'))
res

tensor([[9.9999e-01, 7.5673e-06],
        [1.0000e+00, 5.2504e-07],
        [1.0000e+00, 6.6455e-08],
        [1.0000e+00, 7.8966e-09],
        [1.0000e+00, 1.1144e-09],
        [1.0000e+00, 3.2789e-10],
        [1.0000e+00, 3.1412e-10],
        [1.0000e+00, 8.5965e-11],
        [1.0000e+00, 1.2559e-10],
        [1.0000e+00, 1.0585e-10]], device='cuda:0', grad_fn=<SoftmaxBackward0>)